In [116]:
import pandas as pd
import pyodbc
from sqlpush.sqlstd import *
from sqlpush.config import *
from tabulate import tabulate

In [49]:
# how to fuzzy match : 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
fuzz.token_sort_ratio("Catherine Gitau M.", "Gitau Catherine")
fuzz.partial_ratio("Catherine M. Gitau","Catherine Gitau")

94

In [35]:
# Prepare connection
cursor, conn = connectdb(config)

In [95]:
# let's define all the needed functions

def getdata(cursor):
    query = "SELECT * FROM belgium_addresses "
    cursor.execute(query)
    tablecol = [i[0] for i in cursor.fetchall()]
    return tablecol


def get_street_id_from_postcode(cursor, postcode):
    query = "SELECT street_id FROM belgium_streets WHERE postcode = '%s'" %(postcode)
    cursor.execute(query)
    tablecol = [i[0] for i in cursor.fetchall()]
    return tablecol

def get_language(streetname):
    streetname = streetname.lower()
    language = "None"
    if "straat" in streetname :
        language = "NL"
    elif "laan" in streetname :
        language = "NL"
    elif "rue" in streetname :
        language = "FR"
    elif "avenue" in streetname :
        language = "FR"
    elif "boulevard" in streetname :
        language = "FR"
    elif "square " in streetname :
        language = "FR"
    elif "allées" in streetname :
        language = "FR"
    else : 
        language = "None"
    return language

def get_street_names_from_postcode_lg(cursor, postcode, language):
    if language == "FR" :
        query = "SELECT streetname_fr FROM belgium_streets WHERE postcode = '%s'" %(postcode)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
    elif language == "NL" :
        query = "SELECT streetname_nl FROM belgium_streets WHERE postcode = '%s'" %(postcode)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
    else : 
        query = "SELECT streetname_fr, streetname_nl FROM belgium_streets WHERE postcode = '%s'" %(postcode)
        cursor.execute(query)
        tablecol = cursor.fetchall()
    return tablecol

# I need a function that will take a list of names as input and a search term
# will build another list with fuzzy match score, and return the value with the highest score
def get_best_matching(list_of_streets, term) :
    scores = []
    for element in list_of_streets : 
        scores.append(fuzz.token_sort_ratio(element, term))
    return scores

# takes as input a 2D list, outupts a 1D list
def transform_list(street_n_list) : 
    total_list = []
    for element in street_n_list : 
        total_list.append(element[0])
        total_list.append(element[1])

def get_best_address_given_scores(scores, list_of_streets):
    max_value = max(scores) 
    max_index = scores. index(max_value)
    return list_of_streets[max_index]

def get_street_ids_give_name(street_name, language) :
    if language == "FR" :
        query = "SELECT street_id FROM belgium_streets WHERE streetname_fr = '%s'" %(street_name)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
        return tablecol
    elif language == "NL" : 
        query = "SELECT street_id FROM belgium_streets WHERE streetname_nl = '%s'" %(street_name)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
        return tablecol
    else :  
        query = "SELECT street_id FROM belgium_streets WHERE streetname_nl = '%s' OR streetname_fr = '%s'" %(street_name)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
        return tablecol

def get_address_info(street_id, house_number) : 
    query = "SELECT * FROM belgium_addresses WHERE street_id = '%s' AND house_number = '%s' AND status <> 'retired'" %(street_id, house_number)
    cursor.execute(query)
    tablecol = cursor.fetchall()
    return tablecol

def get_address_info_wou_housenb(street_id) : 
    query = "SELECT * FROM belgium_addresses WHERE street_id = '%s' AND status <> 'retired'" %(street_id)
    cursor.execute(query)
    tablecol = cursor.fetchall()
    return tablecol

In [99]:
# let's implement the algorithm

# take the inputs : postcode, streetname and house number
postcode = input("Ener postcode : ")
streetname = input("Enter streetname : ")
house_number = input("Enter house number : ")

In [100]:
street_ids = get_street_id_from_postcode(cursor, "1040")

In [101]:
best_street_match = ""
language = get_language(streetname)
if language == "None" : 
    sn_none = get_street_names_from_postcode_lg(cursor, postcode, language)
    sn_none = transform_list(sn_none)
    scores = get_best_matching(sn_none, streetname)
    best_street_match = get_best_address_given_scores(scores, sn_none)
else : 
    sn = get_street_names_from_postcode_lg(cursor, postcode, language)
    scores = get_best_matching(sn, streetname)
    best_street_match = get_best_address_given_scores(scores, sn)
street_ids = get_street_ids_give_name(best_street_match, language)

In [119]:
info = get_address_info(street_ids[0], house_number)    
if len(info) == 1 : 
    # best case
    coordinates_x = info[0]
    coordinates_y = info[1]
    print("Here are the coordinates : ")
    print(coordinates_x)
    print(coordinates_y)
    col_names = ["coord X", "coord Y", "lat", "long", "address id", "box_number", "housenumber", "municipality_id", "municipality_name_de", "municipality_name_fr", "municipality_name_nl", "postcode", "postname_fr", "postname_nl", "street_id", "streetname_de", "streetname_fr", "streetname_nl", "region_code", "status"]
    print(tabulate(info, headers=col_names, tablefmt="fancy_grid"))
elif len(info) == 0 : 
    print("No data was found given the street id and house number, but here is what we can find without the house number : ")
    info_wo = get_address_info_wou_housenb(street_ids[0])
    col_names = ["coord X", "coord Y", "lat", "long", "address id", "box_number", "housenumber", "municipality_id", "municipality_name_de", "municipality_name_fr", "municipality_name_nl", "postcode", "postname_fr", "postname_nl", "street_id", "streetname_de", "streetname_fr", "streetname_nl", "region_code", "status"]
    print(tabulate(info_wo, headers=col_names, tablefmt="fancy_grid"))
else : 
    print("Unusual result, but let's print it out : ")
    col_names = ["coord X", "coord Y", "lat", "long", "address id", "box_number", "housenumber", "municipality_id", "municipality_name_de", "municipality_name_fr", "municipality_name_nl", "postcode", "postname_fr", "postname_nl", "street_id", "streetname_de", "streetname_fr", "streetname_nl", "region_code", "status"]
    print(tabulate(info, headers=col_names, tablefmt="fancy_grid"))

No data was found given the street id and house number, but here is what we can find without the house number : 
╒═══════════╤═══════════╤═════════╤═════════╤══════════════╤══════════════╤═══════════════╤═══════════════════╤════════════════════════╤════════════════════════╤════════════════════════╤════════════╤════════════════════════════════════════════════════╤══════════════════════════════════════════════════╤═════════════╤═════════════════╤═════════════════╤═════════════════╤═══════════════╤══════════╕
│   coord X │   coord Y │     lat │    long │   address id │ box_number   │   housenumber │   municipality_id │ municipality_name_de   │ municipality_name_fr   │ municipality_name_nl   │   postcode │ postname_fr                                        │ postname_nl                                      │   street_id │ streetname_de   │ streetname_fr   │ streetname_nl   │ region_code   │ status   │
╞═══════════╪═══════════╪═════════╪═════════╪══════════════╪══════════════╪══════════════